# TNT Multi-Species: Flow Curve Fitting

**Objectives:**
- Fit TNT multi-species model to steady shear flow curve data
- Understand multiple independent bond species with different lifetimes
- Decompose total stress into per-species contributions
- Analyze discrete relaxation spectrum
- Compare NLSQ and Bayesian inference

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt
import arviz as az

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTMultiSpecies

sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_ml_ikh_flow_curve,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_multi_species_param_names,
    plot_multi_species_spectrum,
    plot_mode_decomposition,
)

## Theory: Multiple Independent Bond Species

The TNT multi-species model describes materials with multiple independent bond populations, each with its own modulus $G_i$ and lifetime $\tau_{b,i}$.

**Total stress:**
$$\sigma = \sum_{i=0}^{N-1} \sigma_i + \eta_s \dot{\gamma}$$

**Per-species contribution (steady shear):**
$$\sigma_i = \frac{G_i \tau_{b,i} \dot{\gamma}}{1 + (\tau_{b,i} \dot{\gamma})^2}$$

**Key physics:**
- Fast bonds (short $\tau_{b,0}$): Dominate high shear rates
- Slow bonds (long $\tau_{b,1}$): Dominate low shear rates
- Discrete relaxation spectrum with 2 modes
- Stress superposition from independent populations

**Parameters (2 species):**
- $G_0$: Fast species modulus [Pa]
- $\tau_{b,0}$: Fast species bond lifetime [s]
- $G_1$: Slow species modulus [Pa]
- $\tau_{b,1}$: Slow species bond lifetime [s]
- $\eta_s$: Solvent viscosity [Pa·s]

## Load Data

In [ ]:
gamma_dot, sigma = load_ml_ikh_flow_curve("ARES_up")

print(f"Data points: {len(gamma_dot)}")
print(f"Shear rate range: {gamma_dot.min():.4f} to {gamma_dot.max():.2f} s^-1")
print(f"Stress range: {sigma.min():.2f} to {sigma.max():.2f} Pa")

fig, ax = plt.subplots(figsize=(8, 6))
ax.loglog(gamma_dot, sigma, 'o', label='Data', markersize=8)
ax.set_xlabel(r'Shear Rate $\dot{\gamma}$ [s$^{-1}$]', fontsize=12)
ax.set_ylabel(r'Stress $\sigma$ [Pa]', fontsize=12)
ax.set_title('Carbopol Flow Curve', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()
plt.close('all')

## NLSQ Fitting

In [ ]:
model = TNTMultiSpecies(n_species=2)
param_names = get_tnt_multi_species_param_names(n_species=2)
print(f"Parameters: {param_names}")

start_time = time.time()
model.fit(gamma_dot, sigma, test_mode="flow_curve", method="scipy")
nlsq_time = time.time() - start_time

print(f"\nNLSQ fit completed in {nlsq_time:.2f} s")
print("\nFitted parameters:")
for name in param_names:
    print(f"  {name}: {model.parameters.get_value(name):.6e}")

In [ ]:
gamma_dot_pred = np.logspace(np.log10(gamma_dot.min()), np.log10(gamma_dot.max()), 200)
sigma_pred = model.predict(gamma_dot_pred, test_mode="flow_curve")

fit_metrics = compute_fit_quality(sigma, model.predict(gamma_dot, test_mode="flow_curve"))
print("\nFit quality:")
print(f"  R² = {fit_metrics['R2']:.6f}")
print(f"  RMSE = {fit_metrics['RMSE']:.6e}")
print(f"  NRMSE = {fit_metrics['NRMSE']:.6f}")

fig, ax = plt.subplots(figsize=(8, 6))
ax.loglog(gamma_dot, sigma, 'o', label='Data', markersize=8, alpha=0.7)
ax.loglog(gamma_dot_pred, sigma_pred, '-', label='NLSQ Fit', linewidth=2)
ax.set_xlabel(r'Shear Rate $\dot{\gamma}$ [s$^{-1}$]', fontsize=12)
ax.set_ylabel(r'Stress $\sigma$ [Pa]', fontsize=12)
ax.set_title(f'TNT Multi-Species Flow Curve (R² = {fit_metrics["R2"]:.4f})', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()
plt.close('all')

## Physical Analysis: Per-Species Stress Decomposition

In [ ]:
fig = plot_mode_decomposition(model, gamma_dot_pred, "flow_curve")
plt.show()
plt.close('all')

print("\nSpecies characteristics:")
G_0 = model.parameters.get_value('G_0')
tau_b_0 = model.parameters.get_value('tau_b_0')
G_1 = model.parameters.get_value('G_1')
tau_b_1 = model.parameters.get_value('tau_b_1')

print(f"\nSpecies 0 (fast):")
print(f"  G_0 = {G_0:.3e} Pa")
print(f"  tau_b_0 = {tau_b_0:.3e} s")
print(f"  Crossover rate ~ 1/tau_b_0 = {1/tau_b_0:.3e} s^-1")

print(f"\nSpecies 1 (slow):")
print(f"  G_1 = {G_1:.3e} Pa")
print(f"  tau_b_1 = {tau_b_1:.3e} s")
print(f"  Crossover rate ~ 1/tau_b_1 = {1/tau_b_1:.3e} s^-1")

print(f"\nLifetime ratio: tau_b_1/tau_b_0 = {tau_b_1/tau_b_0:.2f}")
print(f"Modulus ratio: G_1/G_0 = {G_1/G_0:.2f}")

## Discrete Relaxation Spectrum

In [ ]:
fig = plot_multi_species_spectrum(model)
plt.show()
plt.close('all')

print("\nSpectrum interpretation:")
print("- Two discrete relaxation times from two bond species")
print("- Fast species contributes at short times/high frequencies")
print("- Slow species contributes at long times/low frequencies")
print("- No continuous distribution (discrete modes only)")

## Important Note on Model Applicability

**Carbopol is not a multi-species network.** It is a single cross-linked polymer gel with a continuous relaxation spectrum. This notebook demonstrates:
1. The multi-species workflow
2. Species decomposition analysis tools
3. How to interpret discrete spectra

Multi-species models are appropriate for:
- Binary blends (two polymer types)
- Multi-length-scale networks
- Materials with distinct fast/slow relaxation mechanisms

For Carbopol, single-species TNT models are more physically appropriate.

## Bayesian Inference

In [ ]:
# CI mode: Skip Bayesian inference to avoid JIT compilation timeout
# Set CI_MODE=1 environment variable to skip
CI_MODE = os.environ.get("CI_MODE", "0") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if CI_MODE:
    print("CI_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run without CI_MODE environment variable")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    result_bayes = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    result_bayes = model.fit_bayesian(
        gamma_dot, sigma,
        test_mode='flow_curve',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
# Skip convergence diagnostics in CI mode
if not CI_MODE:
    print_convergence_summary(result_bayes, param_names)
else:
    print("CI_MODE: Skipping convergence diagnostics")


## ArviZ Diagnostics: Trace Plots

In [ ]:
# Skip trace plot in CI mode
if not CI_MODE:
    idata = az.from_dict(posterior={name: result_bayes.posterior_samples[name][None, :] for name in param_names})
    axes = az.plot_trace(idata, var_names=param_names, compact=True)
    plt.tight_layout()
    plt.show()
    plt.close('all')
else:
    print("CI_MODE: Skipping trace plot")


## ArviZ Diagnostics: Posterior Distributions

In [ ]:
# Skip ArviZ plot in CI mode
if not CI_MODE:
    fig = az.plot_posterior(idata, var_names=param_names, hdi_prob=0.95)
    plt.tight_layout()
    plt.show()
    plt.close('all')
else:
    print("CI_MODE: Skipping ArviZ plot")


## ArviZ Diagnostics: Pair Plot

In [ ]:
# Skip pair plot in CI mode
if not CI_MODE:
    axes = az.plot_pair(idata, var_names=param_names, divergences=True)
    plt.show()
    plt.close('all')
else:
    print("CI_MODE: Skipping pair plot")


## NLSQ vs Bayesian Parameter Comparison

In [ ]:
print_parameter_comparison(model, result_bayes.posterior_samples, param_names)

## Posterior Predictive: Flow Curve

In [ ]:
posterior = result_bayes.posterior_samples
n_draws = min(200, NUM_SAMPLES)
draw_indices = np.linspace(0, NUM_SAMPLES - 1, n_draws, dtype=int)

x_pred = jnp.array(gamma_dot_pred)
y_pred_samples = []

for i in draw_indices:
    params_i = jnp.array([posterior[name][i] for name in param_names])
    y_pred_i = model.model_function(x_pred, params_i, test_mode="flow_curve")
    y_pred_samples.append(np.array(y_pred_i))

y_pred_samples = np.array(y_pred_samples)
y_pred_mean = np.mean(y_pred_samples, axis=0)
y_pred_lower = np.percentile(y_pred_samples, 2.5, axis=0)
y_pred_upper = np.percentile(y_pred_samples, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(8, 6))
ax.loglog(gamma_dot, sigma, 'o', label='Data', markersize=8, alpha=0.7, zorder=3)
ax.loglog(gamma_dot_pred, y_pred_mean, '-', label='Posterior Mean', linewidth=2, zorder=2)
ax.fill_between(gamma_dot_pred, y_pred_lower, y_pred_upper, alpha=0.3, label='95% CI', zorder=1)
ax.set_xlabel(r'Shear Rate $\dot{\gamma}$ [s$^{-1}$]', fontsize=12)
ax.set_ylabel(r'Stress $\sigma$ [Pa]', fontsize=12)
ax.set_title('Posterior Predictive: Flow Curve', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()
plt.close('all')

## Physical Interpretation

**Multi-species stress contributions:**
- Fast species dominates at high shear rates (short timescale)
- Slow species dominates at low shear rates (long timescale)
- Crossover region shows mixed contributions

**Discrete spectrum:**
- Two well-separated relaxation times
- No intermediate modes (discrete, not continuous)
- Lifetime ratio indicates timescale separation

**Uncertainty quantification:**
- Bayesian posterior captures parameter correlations
- Predictive uncertainty from finite data
- Species parameters may trade off in fitting

## Save Results

In [ ]:
save_tnt_results(model, result_bayes, "multi_species", "flow_curve", param_names)
print("Results saved successfully.")

## Key Takeaways

1. **Multi-species framework**: Independent bond populations with different lifetimes
2. **Stress superposition**: Total stress is sum of species contributions
3. **Timescale separation**: Fast/slow species dominate different shear rate regimes
4. **Discrete spectrum**: Finite number of relaxation modes
5. **Workflow demonstration**: This notebook shows analysis tools, not physical appropriateness for Carbopol
6. **Bayesian inference**: Quantifies parameter uncertainty and correlations
7. **Species decomposition**: Visualization tools reveal individual contributions